# Data Export

This notebook outlines browsing of the database and use of the `pipeline.export` module to export recording data.

The data format is a custom format used internally by project collaborators to support non-datajoint code, and supports export of processed per-probe electrophysiology data along with the experimental events pertaining to that data.

If a session is known, the export capability is also available from the `mapshell.py` script; this usage will be documented below.

In [2]:
from pipeline import lab
from pipeline import experiment
from pipeline import ephys
from pipeline import export

Connecting thinh@mesoscale-activity.datajoint.io:3306


## Find Session / Probe Insertion

In [3]:
experiment.Session & ephys.ProbeInsertion

subject_id institution 6 digit animal ID,session session number,session_date,session_time,username,rig
412330,64,2018-07-16,15:46:24,daveliu,RRig
412753,89,2018-07-10,14:19:59,daveliu,RRig
413950,120,2018-10-22,10:47:29,daveliu,RRig
413950,121,2018-10-23,10:52:16,daveliu,RRig
432570,13,2018-10-22,14:06:50,daveliu,RRig
432572,9,2018-11-20,20:42:01,daveliu,RRig
432572,10,2018-11-21,19:14:39,daveliu,RRig


In [4]:
session_key = {'subject_id': 432572, 'session': 9}

In [5]:
ephys.ProbeInsertion & session_key

subject_id institution 6 digit animal ID,session session number,insertion_number,probe unique identifier for this model of probe (e.g. part number),probe_type e.g. neuropixels_1.0,electrode_config_name user friendly name
432572,9,1,641250222,neuropixels 1.0 - 3A,1-384
432572,9,2,641251247,neuropixels 1.0 - 3A,1-384


The selected session has 2 probe insertions, here we can select just one for export

In [6]:
insertion_key = {**session_key, 'insertion_number': 1}

## Export Data

The `export.export_recording` function is used to export data. Online help is available as follows

In [5]:
help(export.export_recording)

Help on function export_recording in module pipeline.export:

export_recording(insert_keys, output_dir='./', filename=None)
    Export a 'recording' (or a list of recording) (probe specific data + related events) to a file.
    
    Parameters:
    
      - insert_keys: one or a list of ephys.ProbeInsertion.primary_key
        currently: {'subject_id', 'session', 'insertion_number'})
    
      - output_dir: directory to save the file at (default to be the current working directory)
    
      - filename: an optional output file path string. If not provided,
        filename will be autogenerated using the 'mkfilename'
        function.
        Note: if exporting a list of probe keys, filename will be auto-generated



In [6]:
help(export.mkfilename)

Help on function mkfilename in module pipeline.export:

mkfilename(insert_key)
    create a filename for the given insertion key.
    filename will be of the format map-export_h2o_YYYYMMDD_HHMMSS_SN_PN.mat
    
    where:
    
      - h2o: water restriction number
      - YYYYMMDD_HHMMSS: session recording datetime
      - SN: session number for this subject
      - PN: probe number for this session



In [7]:
export.export_recording(insertion_key)

exporting {'subject_id': 432572, 'session': 9, 'session_date': '2018-11-20', 'insertion_number': 1} to map-export_dl56_20181120_204201_s9_p1.mat
fetching spike/behavior data
reshaping/processing for export
... neuron_single_units:ok.
... neuron_unit_info:

C:\Users\thinh\AppData\Roaming\Python\Python37\site-packages\datajoint\connection.py:211: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


ok.
... behavior_report:ok.
... behavior_early_report:ok.
... behavior_lick_times:ok.
... task_trial_type:ok.
... task_stimulation:ok.
... task_cue_time:ok.
... saving to map-export_dl56_20181120_204201_s9_p1.mat:ok.


## Export Data (pipeline.shell - interactive)

The `pipeline.shell` module wraps the export_recording fuction - usage is as above:

```
from pipeline import shell
shell.export_recording(insertion_key)
```

## Export Data (mapshell.py)

To use the `export-recording` feature of the `mapshell.py` convenience script, pass in a quoted key and optional filename:


```
$ mapshell.py export-recording "{'subject_id': 435884, 'session': 1, 'insertion_number': 1}"
```

### Note: `export.export_recording()` can also work on a list of `insertion_key` as input, for batch export

In [12]:
insertion_keys = (ephys.ProbeInsertion & session_key).fetch('KEY')

C:\Users\thinh\AppData\Roaming\Python\Python37\site-packages\datajoint\connection.py:211: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [13]:
insertion_keys

[{'subject_id': 432572, 'session': 9, 'insertion_number': 1},
 {'subject_id': 432572, 'session': 9, 'insertion_number': 2}]

In [ ]:
export.export_recording(insertion_keys)


## Restricted by the `ephys.ProbeInsertionQuality` table

The `ephys.ProbeInsertionQuality` stores users' manual labeling of whether or not 
a particular ProbeInsertion is of good quality

To restrict the export to only insertions that are labeled as good, you'll just need to add
this restriction to your session/probe-insertion query

In [ ]:
insertion_keys = (ephys.ProbeInsertion
                  & (ephys.ProbeInsertionQuality & 'drift_presence = 0')
                  & session_key).fetch('KEY')

export.export_recording(insertion_keys)